[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/alesaccoia/IULM_DDM2324_Notebooks/blob/main/10_dentmax_hierarchical_clustering.ipynb)

# Capitolo 5: Dentmax, Exploratory Data Analysis

In [ ]:
# Import necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import pearsonr

# Read the data
df = pd.read_excel("data/Capitolo5e6-DentMax.xlsx")

# Display the first few rows of the dataframe
df.head()


In [ ]:
df.info()

In [ ]:
df.describe()

Standardizzazione delle variabili

In [ ]:
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df.iloc[:, 1:])
df_scaled

In [ ]:
# Clustering gerarchico
Z = linkage(df_scaled, method='ward', metric='euclidean')

In [ ]:
# Visualizza il dendrogramma
plt.figure(figsize=(12, 6))
dendrogram(Z, p=10, truncate_mode='level')
plt.xlabel('Nodi')
plt.ylabel('Distanza')
plt.show()

# Decisione dell'altezza di "taglio"

Dopo aver esaminato il dendrogramma, é possibile valre quali e quanti gruppi tenere. Anche se é difficile trovare l'altezza giusta dal grafico precedente, é possibile comunque chiedere al programma quanti cluster rimangono dopo aver "tagliato" il dendrogramma a una certa altezza. Proviamo con 75, per ottenere 4 cluster

In [ ]:
threshold = 75

In [ ]:
# Effettua il clustering in base all'altezza di taglio scelta
clusters = fcluster(Z, t=threshold, criterion='distance')
np.unique(clusters)

75 pare un'altezza corretta e rileviamo 4 cluster (la variabile contiene gli ID ordinati, uno per osservazione)

In [ ]:
clusters

Aggiungiamo la colonna con l'ID del cluster al cluster originale, e non a quello normalizzato.
In tal modo abbiamo operato su dati normalizzati, ma interpreteremo dati sulle loro scale reali

In [ ]:
# Aggiungi la colonna dei cluster al DataFrame originale (non a quello normalizzato, in tal modo )
df['Cluster'] = clusters

In [ ]:
# Visualizza le differenze tra i cluster mediante boxplot
plt.figure(figsize=(8, 6))
sns.boxplot(x='Cluster', y='Historical Sales Index', data=df)
plt.xlabel('Cluster')
plt.ylabel('Valutazione')
plt.show()


## Proporzione per cluster

In [ ]:
cluster_counts = df['Cluster'].value_counts()
proportions = cluster_counts / df.shape[0]

print(proportions)

In [ ]:
# Calculate centroids for each cluster
cluster_centroids = df.groupby('Cluster').mean().reset_index()
print(cluster_centroids)



## Trasposizione dei cluster per visualizzazione tabella

In [ ]:

# Calculate centroids for each cluster
cluster_centroids = cluster_centroids.round(2)

# Transpose the matrix
cluster_centroids_transposed = cluster_centroids.transpose()

# Correct the column headers after transposition
cluster_centroids_transposed.columns = cluster_centroids_transposed.iloc[0]
cluster_centroids_transposed = cluster_centroids_transposed.drop('Cluster')
cluster_centroids_transposed



In [ ]:
cluster_centroids_transposed.to_csv('cluster_centroids_transposed.csv')

## Visualizzazione con PCA

In [ ]:
import seaborn as sns

from sklearn.decomposition import PCA

# Esegui PCA
pca = PCA(n_components=2)
df_pca = pca.fit_transform(df_scaled)

# Set the style of the plot
sns.set(style="whitegrid")

# Create a scatter plot of the PCA results with colors based on cluster ID
plt.figure(figsize=(10, 6))
sns.scatterplot(x=df_pca[:, 0], y=df_pca[:, 1], hue=df['Cluster'], palette='tab10', s=80)
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.legend(title='Cluster', loc='best', labels=['Cluster ' + str(i) for i in range(1, len(np.unique(clusters)) + 1)])
plt.grid(False)
plt.show()